In [1]:
from langchain_community.document_loaders import Docx2txtLoader, TextLoader, PyPDFLoader
import os
import inspect


print(inspect.getsourcefile(Docx2txtLoader))
print(inspect.getsourcefile(TextLoader))

d:\anaconda3\envs\RAG_KNOWLEDGE_QA\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


d:\anaconda3\envs\RAG_KNOWLEDGE_QA\Lib\site-packages\langchain_community\document_loaders\word_document.py
d:\anaconda3\envs\RAG_KNOWLEDGE_QA\Lib\site-packages\langchain_community\document_loaders\text.py


In [2]:
from langchain_chroma import Chroma
print(inspect.getsourcefile(Chroma))

d:\anaconda3\envs\RAG_KNOWLEDGE_QA\Lib\site-packages\langchain_chroma\vectorstores.py


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

print(inspect.getsourcefile(RecursiveCharacterTextSplitter))

d:\anaconda3\envs\RAG_KNOWLEDGE_QA\Lib\site-packages\langchain_text_splitters\character.py


In [ ]:
from serpapi import GoogleSearch
SERPAPI_KEY = os.environ.get("SERPAPI_KEY")
params = {
  "engine": "google_light",
  "q": "美国总统是谁，介绍一下他",
  "google_domain": "google.com",
  "hl": "zh-cn",
  "gl": "cn",
  "api_key": SERPAPI_KEY,
}

search = GoogleSearch(params)
results = search.get_dict()
organic_results = results["organic_results"]

In [2]:
organic_results

[{'position': 1,
  'title': '美国总统- 维基百科，自由的百科全书',
  'link': 'https://zh.wikipedia.org/zh-cn/%E7%BE%8E%E5%9B%BD%E6%80%BB%E7%BB%9F',
  'displayed_link': 'zh.wikipedia.org › zh-cn › 美国总统',
  'snippet': '作为当今世界唯一超级大国的国家元首与政府首脑，其号称“自由世界的领导者”及“世界上最有权力的人”。现任（第47任）美国总统是唐纳德·特朗普，于2025年1月20日上任。 美利坚 ...',
  'sitelinks': {'inline': [{'title': '列表',
     'link': 'https://zh.wikipedia.org/zh-cn/%E7%BE%8E%E5%9B%BD%E6%80%BB%E7%BB%9F%E5%88%97%E8%A1%A8'},
    {'title': '美国总统选举',
     'link': 'https://zh.wikipedia.org/zh-cn/%E7%BE%8E%E5%9B%BD%E6%80%BB%E7%BB%9F%E9%80%89%E4%B8%BE'},
    {'title': '支持率',
     'link': 'https://zh.wikipedia.org/zh-cn/%E7%BE%8E%E5%9B%BD%E6%80%BB%E7%BB%9F%E6%94%AF%E6%8C%81%E7%8E%87'},
    {'title': '总统当选人',
     'link': 'https://zh.wikipedia.org/zh-cn/%E7%BE%8E%E5%9B%BD%E6%80%BB%E7%BB%9F%E5%BD%93%E9%80%89%E4%BA%BA'}]}},
 {'position': 2,
  'title': '唐纳德·特朗普_百度百科',
  'link': 'https://baike.baidu.com/item/%E5%94%90%E7%BA%B3%E5%BE%B7%C2%B7%E7%89%B9%E6%9C%97%E6%99%AE/9916449',
  'dis

In [2]:
import inspect
from langchain_chroma import Chroma
print(inspect.getsourcefile(Chroma))

d:\anaconda3\envs\RAG_KNOWLEDGE_QA\Lib\site-packages\langchain_chroma\vectorstores.py
